In [1]:
from tensorflow import keras
import pandas as pd
from sklearn.model_selection import train_test_split

2023-11-29 15:05:52.069090: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = pd.read_csv('data/output/matches.csv')
data.head()

,winner,pokemon1_p1,pokemon2_p1,pokemon3_p1,pokemon4_p1,pokemon5_p1,pokemon6_p1,pokemon1_p2,pokemon2_p2,pokemon3_p2,pokemon4_p2,pokemon5_p2,pokemon6_p2
0,1,Dondozo,Tatsugiri,Ogerpon,Iron Hands,Ninetales,Glimmora,Weezing,Landorus,Ogerpon,Heatran,Rillaboom,Iron Hands
1,1,Tornadus,Sinistcha,Basculegion,Iron Hands,Arcanine,Ninetales,Tornadus,Arcanine,Gastrodon,Rillaboom,Chi,Iron Valiant
2,2,Urshifu,Landorus,Ogerpon,Thundurus,Rillaboom,Kingambit,Armarouge,Indeedee,Torkoal,Ursaluna,Gallade,Urshifu
3,2,Gholdengo,Roaring Moon,Rillaboom,Arcanine,Urshifu,Kingambit,Ogerpon,Dondozo,Tatsugiri,Flutter Mane,Glimmora,Urshifu
4,1,Dondozo,Tatsugiri,Rillaboom,Volcarona,Kingambit,Scizor,Arcanine,Rillaboom,Tornadus,Flutter Mane,Urshifu,Chien


In [3]:
vocabulary = set()

for col in data.columns:
    for value in data[col]:
        if isinstance(value, str):
            vocabulary.add(value)
vocabulary = list(vocabulary)
print('Vocabulary size:', len(vocabulary))
print('Vocabulary:', vocabulary)

Vocabulary size: 392
Vocabulary: ['Electrode', 'Flamigo', 'Venonat', 'Ogerpon', 'Salamence', 'Lurantis', 'Slowking', 'Pawniard', 'Magcargo', 'Impidimp', 'Koffing', 'Dusclops', 'Spiritomb', 'Trevenant', 'Kingambit', 'Lumineon', 'Snorlax', 'Walking Wake', 'Pincurchin', 'Espathra', 'Heatran', 'Qwilfish', 'Basculegion', 'Volcarona', 'Ting', 'Kricketune', 'Toedscool', 'Hydreigon', 'Bronzor', 'Charjabug', 'Abomasnow', 'Thundurus', 'Dusknoir', 'Farigiraf', 'Mareep', 'Great Tusk', 'Ursaluna', 'Mienshao', 'Tauros', 'Ariados', 'Masquerain', 'Medicham', 'Numel', 'Chesnaught', 'Gliscor', 'Makuhita', 'Leafeon', 'Kilowattrel', 'Iron Bundle', 'Cinderace', 'Murkrow', 'Veluza', 'Delphox', 'Ursaring', 'Cleffa', 'Stonjourner', 'Salazzle', 'Gallade', 'Armarouge', 'Iron Thorns', 'Eelektross', 'Whiscash', 'Talonflame', 'Glaceon', 'Kommo', 'Yanma', 'Gothitelle', 'Iron Jugulis', 'Braviary', 'Haxorus', 'Brambleghast', 'Mandibuzz', 'Altaria', 'Dragonite', 'Skeledirge', 'Dragalge', 'Mudsdale', 'Ninetales', 'Gree

In [4]:
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [5]:
X_train = train.drop(columns=['winner'])
y_train = train['winner'] - 1

X_test = test.drop(columns=['winner'])
y_test = test['winner'] - 1

In [6]:
judge = keras.Sequential([
    keras.layers.StringLookup(vocabulary=vocabulary, mask_token=None),
    # keras.layers.Embedding(input_dim=len(vocabulary) + 1, output_dim=6, input_length=6),
    keras.layers.Embedding(input_dim=len(vocabulary) + 1, output_dim=64, input_length=6),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation='relu'),
    # keras.layers.Dense(4, activation='relu'),
    keras.layers.Dense(1)
])

t1 = keras.Input(shape=(6,), dtype='string')
t2 = keras.Input(shape=(6,), dtype='string')

s1 = judge(t1)
s2 = judge(t2)

d = keras.layers.Subtract()([s1, s2])
# d = keras.layers.Activation('sigmoid')(d)

model = keras.Model(inputs=[t1, t2], outputs=d)

In [7]:
model.compile(
    optimizer='adam',
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [8]:
X_train_p1 = X_train[['pokemon1_p1', 'pokemon2_p1', 'pokemon3_p1', 'pokemon4_p1', 'pokemon5_p1', 'pokemon6_p1']].to_numpy()
X_train_p2 = X_train[['pokemon1_p2', 'pokemon2_p2', 'pokemon3_p2', 'pokemon4_p2', 'pokemon5_p2', 'pokemon6_p2']].to_numpy()

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 sequential (Sequential)        (None, 1)            26209       ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 subtract (Subtract)            (None, 1)            0           ['sequential[0][0]',         

In [10]:
model.fit([X_train_p1, X_train_p2], y_train, epochs=200, verbose=2, batch_size=32, validation_split=0.2)

Epoch 1/200


288/288 - 1s - loss: 0.6871 - accuracy: 0.5035 - val_loss: 0.6784 - val_accuracy: 0.5169 - 1s/epoch - 5ms/step
Epoch 2/200
288/288 - 0s - loss: 0.6661 - accuracy: 0.5385 - val_loss: 0.6740 - val_accuracy: 0.5209 - 382ms/epoch - 1ms/step
Epoch 3/200
288/288 - 0s - loss: 0.6523 - accuracy: 0.5598 - val_loss: 0.6736 - val_accuracy: 0.5395 - 382ms/epoch - 1ms/step
Epoch 4/200
288/288 - 0s - loss: 0.6431 - accuracy: 0.5731 - val_loss: 0.6720 - val_accuracy: 0.5421 - 374ms/epoch - 1ms/step
Epoch 5/200
288/288 - 0s - loss: 0.6358 - accuracy: 0.5826 - val_loss: 0.6746 - val_accuracy: 0.5469 - 375ms/epoch - 1ms/step
Epoch 6/200
288/288 - 0s - loss: 0.6294 - accuracy: 0.5899 - val_loss: 0.6753 - val_accuracy: 0.5482 - 375ms/epoch - 1ms/step
Epoch 7/200
288/288 - 0s - loss: 0.6243 - accuracy: 0.5954 - val_loss: 0.6748 - val_accuracy: 0.5560 - 370ms/epoch - 1ms/step
Epoch 8/200
288/288 - 0s - loss: 0.6195 - accuracy: 0.6026 - val_loss: 0.6788 - val_accuracy: 0.5547 - 373ms/epoch - 1ms/step
Epoch 9

In [11]:
X_test_p1 = X_test[['pokemon1_p1', 'pokemon2_p1', 'pokemon3_p1', 'pokemon4_p1', 'pokemon5_p1', 'pokemon6_p1']].to_numpy()
X_test_p2 = X_test[['pokemon1_p2', 'pokemon2_p2', 'pokemon3_p2', 'pokemon4_p2', 'pokemon5_p2', 'pokemon6_p2']].to_numpy()

model.evaluate([X_test_p1,X_test_p2], y_test, verbose=2)

90/90 - 0s - loss: 1.1216 - accuracy: 0.5860 - 118ms/epoch - 1ms/step


[1.1215544939041138, 0.5860270857810974]